In [ ]:
# import all the libraries needed
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tf as tf
from matplotlib.image import imread

In [ ]:
# initializing the train and test paths for the data
# make sure to change this to where you have your data
data_dir = 'C:\\Users\\nagar\\OneDrive\\Documents\\Projects\\CatorDog\\PetImages'
os.listdir(data_dir)

In [ ]:
test_data_path = data_dir+'\\test\\'
training_data_path = data_dir+'\\train\\'

In [ ]:
os.listdir(test_data_path)

In [ ]:
os.listdir(training_data_path)

In [ ]:
# let us see an example image of each a dog and a cat
cat_img = 'C:\\Users\\nagar\\OneDrive\\Documents\\Projects\\CatorDog\\PetImages\\Train\\Cat\\0.jpg'
cat_image = imread(cat_img)
plt.imshow(cat_image)

In [ ]:
dog_img = 'C:\\Users\\nagar\\OneDrive\\Documents\\Projects\\CatorDog\\PetImages\\Train\Dog\\0.jpg'
dog_image = imread(dog_img)
plt.imshow(dog_image)

In [ ]:
# check the shape of each image
dog_image.shape

In [ ]:
cat_image.shape

In [ ]:
# check if all the dimension are the same
dim1 = []
dim2 = []
for image_filename in os.listdir(test_data_path+'\\Dog'):

    img = imread(test_data_path+'\\Dog'+'\\'+image_filename)
    d1,d2,colors = img.shape
    dim1.append(d1)
    dim2.append(d2)

In [ ]:
sns.jointplot(dim1,dim2)

In [ ]:
#calculate the means in order to reshape all the other images
np.mean(dim1)

In [ ]:
np.mean(dim2)

In [ ]:
image_shape = (369,399,3) #x,y,and color levels

In [ ]:
# using keras image generator to add more data for the CNN model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_generator_model = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.10,
                               height_shift_range=0.10,
                               rescale=1/255,
                               shear_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest')

In [ ]:
# test out the image generator on the original dog photo
# original dog model
plt.imshow(dog_image)

In [ ]:
plt.imshow(image_generator_model.random_transform(dog_image))

In [ ]:
# image generation for each folder (training and test)
# see how it doubles the data for each folder
image_generator_model.flow_from_directory(training_data_path)

In [ ]:
image_generator_model.flow_from_directory(test_data_path)

In [ ]:
# creating the CNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization

In [ ]:


model = Sequential()

# add first CNN layer
model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add(BatchNormalization())
# add max pooling after convulution to reduce processing time
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# repeat two more time with more filters
model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#flatten the model for the dense layers
model.add(Flatten())

#add the dense layers with a ReLu activiation
model.add(Dense(128))
model.add(Activation('relu'))

# add dropout to reduce overfitting
model.add(Dropout(0.5))

# add last dense layer that will be the output + use sigmoid activation to classify whether it is a dog or cat
# note: only one nueron becuase it is checking it is a dog or not
model.add(Dense(1))
model.add(Activation('sigmoid'))

# compile the model and add an accuracy metric
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# add an early stopping callback in order to reduce wasting time
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
# using the image generator we made earlier to create train and test sets

training_image_set = image_generator_model.flow_from_directory(training_data_path,target_size=image_shape[:2],color_mode='rgb',batch_size=16,class_mode='binary')

In [ ]:
test_image_set = image_generator_model.flow_from_directory(test_data_path,target_size=image_shape[:2],color_mode='rgb',batch_size=16,class_mode='binary',shuffle=False)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# fit the model
data_fit = model.fit_generator(training_image_set,epochs=20,validation_data=test_image_set,callbacks=[early_stopping])

In [ ]:
# if you just want to use the model and skip the train time then

In [15]:
# from tensorflow.keras.models import load_model
# # model.save('model.h5')
# new_model = load_model('model.h5')

ValueError: No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x0000015FA27B3610>.

In [6]:
#get the metrics for the model --> classification report as well as confusion matrix
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.preprocessing import image

In [ ]:
model.evaluate_generator(test_image_set)

In [ ]:
prediction_prob = model.predict_generator(test_image_set)

In [ ]:
prediction_prob

In [ ]:
predictions = prediction_prob>0.5

In [ ]:
print(classification_report(test_image_set.classes,predictions))

In [ ]:
confusion_matrix(test_imag_set.classes,predictions)